In [101]:
# Start writing code here...

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from igraph import *
import disjoint_set

This notebook contains template with example of data usage and loading (from prepared csv files, which were produced from original .xlsx file with the help of pandas)

In [102]:
if not ".." in sys.path:
    sys.path.insert(0, "..")

In [103]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
os.listdir("../data")

['sources.csv.gz',
 'input_wagon.xlsx',
 'orders7.csv.gz',
 'orders8.csv.gz',
 'wagon_mode_compat.csv.gz',
 'description.txt',
 'output_wagon.xlsx',
 'metrics.csv.gz',
 'standplaces.csv.gz']

In [104]:
orders7 = pd.read_csv("../data/orders7.csv.gz")
orders8 = pd.read_csv("../data/orders8.csv.gz")
wagon_mode_compat = pd.read_csv("../data/wagon_mode_compat.csv.gz")
sources = pd.read_csv("../data/sources.csv.gz")
standplaces = pd.read_csv("../data/standplaces.csv.gz")
metrics = pd.read_csv("../data/metrics.csv.gz")

In [105]:
# load data from file (this gets checked and reloaded because of autoreload magic above)
from utils.common import *



In [46]:
# get_roads_graph


TypeError: '(slice(104200, 104210, None),)' is an invalid key

In [106]:
from igraph import *
from utils.abstractions import get_roads_graph


graph, name2vertex = get_roads_graph(metrics)


0
50000
100000
150000
200000
250000
300000
350000
400000
450000
500000
550000
600000
650000
700000
750000
800000
850000
900000


In [107]:
total_sum = 0

stale_price = {}
stale_move = {}
for index, row in sources.iterrows():
    stale_price[row['Station']] = int(row['Stand']) 


for index, row in metrics.iterrows():
    stale_move[str(row['From']) + "_" + str(row['To'])] = int(row['PriceUnit']) 
    stale_move[str(row['To']) + "_" + str(row['From'])] = int(row['PriceUnit']) 


In [121]:
from utils.common import *

order_manager = OrderManager()
for index, row in orders7.iterrows():
    order = Order2(row['Start'], row['Finish'], row['StartDate'], int(row['MaxUnit']), row['NeedWagonModel'], int(row['Tariff'].split(":")[0]), row['Dur'], row['ShortagePenalty'])
    order_manager.add_order(order)
    total_sum += order.total_profit()
print(total_sum)

wagon_manager = WagonManager()
for index, row in sources.iterrows():
    for units in range(row['Units']):
        occupied = True
        if int(row['Date']) == 0:
            occupied = False
        wagon = Wagon(row['WagonModel'], str(row['Station']), occupied, row['Date'])
        wagon_manager.add_wagon(wagon)

print(wagon_manager.wagon_number())


comp_wagon = set()
for index, row in wagon_mode_compat.iterrows():
    comp_wagon.add(str(row['NeedWagonModel']) + "_" + str(row['CompatibleWagonModel']))



strategy =  Strategy(order_manager, wagon_manager, graph, comp_wagon, name2vertex, stale_price, stale_move)
print(strategy.compability_model)
for day in range(28):
    strategy.run_day(day)
    print(day)
    #break
print("Bad orders - " +  str(strategy.bad_orders))
print("Total orders - " +  str(strategy.total_orders))

print("Profit - " + str(strategy.total_profit))
print("Stale stop - " + str(strategy.stale_stop))
print("Stale move - " + str(strategy.stale_move))
print("Profit per cat - " + str(strategy.profit_per_cat))


NameError: name 'Order2' is not defined

In [95]:
strategy.stale_moves_num

8954

In [120]:
with open("orders.csv", 'w') as orders:
    orders.write("Отправление,Прибытие,Тип простоя,Модель вагона,День отправления (от начала расчета),День прибытия (от начала расчета),Количество,Цена порожнего пробега (за вагон),Код следующего заказa,Расчитанный тариф следующего заказа\n")
    for order in strategy.order_log:
        orders.write(order)
        


AttributeError: 'Strategy' object has no attribute 'order_log'

![Picture title](image-20210222-045846.png)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7ba67d69-ce72-437b-8bf4-6d3c9b906c4a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>